In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/v0.6.3_ext.pth

In [1]:
import os
import io
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython.display import display
from torchvision.transforms import ToPILImage, PILToTensor, Resize
from walloc import walloc
from walloc.walloc import latent_to_pil, pil_to_latent
from datasets import load_dataset, get_dataset_config_names
class Args: pass

In [2]:
device = "cuda"
checkpoint = torch.load("v0.6.3_ext.pth",map_location="cpu")
args = checkpoint['args']
codec = walloc.Walloc(
    channels = args.channels,
    J = args.J,
    N = args.N,
    latent_dim = args.latent_dim,
    latent_bits = 5
)
codec.load_state_dict(checkpoint['model_state_dict'])
codec = codec.to(device)
codec.eval();

In [3]:
config_names = get_dataset_config_names("danjacobellis/imagenet-1k-standardized")
for size in config_names:
    inet1k = load_dataset("danjacobellis/imagenet-1k-standardized",name=size)
    break

In [4]:
inet1k = load_dataset("danjacobellis/imagenet-1k-standardized",name="large_landscape")

In [9]:
def walloc_encode_png(batch):
    with torch.no_grad():
        x = [PILToTensor()(img).unsqueeze(0) for img in batch['image']]
        x = torch.cat(x,dim=0)
        x = x.to(torch.float)
        x = (x/255 - 0.5)
        x = x.to(device)
        X = codec.wavelet_analysis(x,J=codec.J)
        latent = codec.encoder(X).to("cpu")
        image_list = latent_to_pil(latent,5,1)
        buffer_list = []
        for image in image_list:
            buff = io.BytesIO()
            image.save(buff, format='png')
            buffer_list.append(buff.getbuffer())
        return {"image": buffer_list,
                "label": batch['label']}

In [17]:
def walloc_encode_tiff(batch):
    with torch.no_grad():
        x = [PILToTensor()(img).unsqueeze(0) for img in batch['image']]
        x = torch.cat(x,dim=0)
        x = x.to(torch.float)
        x = (x/255 - 0.5)
        x = x.to(device)
        X = codec.wavelet_analysis(x,J=codec.J)
        latent = codec.encoder(X).to("cpu")
        image_list = latent_to_pil(latent,5,4)
        buffer_list = []
        for image in image_list:
            buff = io.BytesIO()
            image.save(buff, format='TIFF', compression="tiff_adobe_deflate")
            buffer_list.append(buff.getbuffer())
        return {"image": buffer_list,
                "label": batch['label']}

In [41]:
walloc_png = inet1k['train'].select(range(4096)).map(walloc_encode_png, batched=True, batch_size=64)
walloc_png.save_to_disk("png")

Saving the dataset (0/1 shards):   0%|          | 0/4096 [00:00<?, ? examples/s]

In [42]:
! du -sh png

229M	png


In [43]:
41*60/229

10.74235807860262

In [44]:
walloc_tiff = inet1k['train'].select(range(4096)).map(walloc_encode_tiff, batched=True, batch_size=64)
walloc_tiff.save_to_disk("tiff")

Saving the dataset (0/1 shards):   0%|          | 0/4096 [00:00<?, ? examples/s]

In [45]:
! du -sh tiff

259M	tiff


In [46]:
50.55*60/259

11.71042471042471